In [7]:
import numpy as np

# 4. 신경망 학습

학습 = 가중치 매개변수를 최적화
그 지표는? 손실함수. (손실함수의 최소화.)
Gradient Descent 를 배울 것이다. 

## 4.1 데이터에서 학습한다!

실제 신경망에서 매개변수 (a,w,b)는 수천 수만개다. 
2장의 퍼셉트론도 선형 분리 가능한 문제는 유한 번 학습을 통해 풀 수 있다. (퍼셉트론 수렴 정리, perceptrain convergence theorem)
하지만 비선형 문제는 불가하다. 

### 4.1.1 데이터 주도 학습

<img src="https://t1.daumcdn.net/cfile/tistory/2137CC4A593E64A224">

단순히 사람이 패턴을 파악해 모델을 설계할 수 없다. 데이터의 특징(feature)을 추출하고, 이를 기계학습 시켜야 한다. 

특징(feature)이란? 입력 데이터에서 본질적인 데이터를 정확하게 추출할 수 있도록 설계된 변환기이다. 
예를 들어 컴퓨터 비전 분야에선 이미지를 벡터로 기술하고 이를 SIFT, SURF, HOG 등의 feature로 뽑는다. 그리고 SVM, KNN등의 기계학습을 돌린다. 
주의할 것은, 문제에 적합한 특징을 설계하는 것은 사람이 하는 일이라는 것이다. 

그런데 신경망(딥러닝)은 이런 것도 필요없다. 있는 그대로를 학습한다. feature까지도 기계가 학습하는 것이다. (종단간 기계학습, end-to-end machine learning: 사람의 개입없이 입력~출력까지 얻는다.)

### 4.1.2 훈련 데이터와 시험 데이터

traning data / test data 분리. 
왜 나누는가? 우리는 범용적인 모델을 원하기 떄문. 아직 보지 못한 데이터도 잘 풀 수 있는지 evaluation이 필요하다. 
overfitting / underfitting 문제. 

## 4.2 손실 함수

주로 
- 평균 제곱 오차 (Mean Square Error)
- 교차 엔트로피 오차 

를 이용한다. 

### 4.2.1 평균 제곱 오차

<img src="https://t1.daumcdn.net/cfile/tistory/241A674E593E6CB206">
yk 는 추정값. (y hat), tk는 정답 레이블 (y), k는 데이터의 차원 수 이다. 

t = [0,0,1,0,...] 로 나와있는데, 이는 one-hot encoding된 정답 label이다. 

여기선 아직 n으로 나눈 (평균) 것을 안한 듯 하다.

In [8]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

### 4.2.2 교차 엔트로피 오차

cross entropy error, CEE
<img src="https://t1.daumcdn.net/cfile/tistory/99C0D73B5A92769625">

tk는 정답일 때만 1인 one-hot encoding되어있는 것이기 때문에 실질적으론 정답일 때의 추정의 자연로그를 계산하는 식.  
맞춘 답을 신경망이 높은 확률로 예측했을 경우 cost가 적고, 낮은 확률로 예측했을 경우 cost가 크다. 

In [9]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta)) # log0이 되지 않게 아주 작은 값 delta를 더해준다. 

### 4.2.3 미니배치 학습

방금 배운 오차를 N개로 확장하여 training data 모두에 대해 구해보고 이를 평균낸다. ( /N) 

이 때, 모든 데이터에 대해 하는 것은 너무 시간이 많이 걸릴 수 있다. 특히 초대규모의 빅데이터라면 더 그렇다. 
이럴 때, 표본을 추출해 근사치로 계산한다. 이 표본을 미니 배치(mini-batch)라고 한다. 

이를 코드로 짜보자. 

In [10]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist 

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

# 이 60000개의 훈련 데이터에서 무작위로 10장만 꺼낸다면?

train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # train_size 미만의 수 중 batch_size만큼 샘플 추출. 
print(batch_mask)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

(60000, 784)
(60000, 10)
[21482 13055 21430 22250  5498  1400 38597 49818 10171 18978]


### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기



## <p style="color:purple">이 부분 무슨 소리인지 잘 모르겠다.. 다시 살펴볼 것. pg.118~pg.119<p>

In [11]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

In [12]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(np.arange(batch_size), t)) / batch_size

### 4.2.5 왜 손실 함수를 설정하는가? 

미분이기 때문에. d손실함수/d매개변수 = (거의)0 으로 만들어야 한다. 
단순히 정확도를 썼을 경우 대부분의 지역에서 0이 되는 문제가 발생한다. 

정확도는 왜 안좋은가? 100장 중 32장을 맞춘 case는 여러 경우가 존재할 수 있다. 매우 이산적인 값을 가진다. (불연속적)
반면 손실함수는 연속적인 값으로 표현된다.  

step function은 대부분에서 기울기가 0이지만, 
sigmoid는 어떤 곳에서도 0이 아니다. 따라서 적절하다. 

## 4.3 수치 미분

미분의 복습.
대부분의 내용은 코드 생략하겠다. 

### 4.3.1 미분

극한을 흉내내서 단순히 수치 미분(numerical differentication) 하는 것은 여러 주의를 요한다. 


1. 반올림 오차(rounding error)문제가 있다. 

    가령 np.float32(1e-50) >> 0.0이 되어버린다. 너무 작은 수이기 때문이다. 0.0001정도는 괜찮다. 
    
2. 극한을 흉내만 내는 것이기 때문에 f(x+h) - f(x) / h 는 어차피 적당하지 않다. 
    
    좀 더 정확히 구하기 위해 x+h와 x 사이를 구하는 전방차분 말고 x+h와 x-h 사이를 쓰는 중심차분을 쓸 수도 있다. 

3. 이런 오차가 없게 하려면 수치 미분이 아닌 해석적(analytic) 미분을 통해 구해줘야 한다. 

### 4.3.2 수치 미분의 예

수치미분을 잘 구현하면 해석적 미분과 큰 차이가 없다. 

### 4.3.3 편미분

생략. 

## 4.4 기울기

모든 변수의 편미분을 백터로 정리한 것을 기울기(gradient)라고 한다. 이는 전미분 개념과 비슷하다고 볼 수 있다. 
이건 한 번 해보겠다. 

In [13]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 shape가 같은 0배열을 생성. 
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원. 
    
    return grad


In [14]:
def function_2(x): #뒤에서 쓰게 될 것 때문에 정의하는 f(x0, x1) = x0^2 + x1^2
    return x[0]**2 + x[1]**2

In [15]:
numerical_gradient(function_2, np.array([3.0, 4.0]))

array([6., 8.])

위 결과의 의미는? 

<img src="https://t1.daumcdn.net/cfile/tistory/99D29B4D5B9CFBB81E">

기울기는 사실 각 지점에서 낮아지는 방향을 가리킨다. 
#### 기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향이다. 

### 4.4.1 경사법(경사하강법)

일반적인 문제의 손실함수는 매우 복잡하다. Gradient Descent는 기울기가 가리키는 방향에 최솟값이 있다고 가정하고 문제를 푸는 것이다. 
문제의 특성에 따라 그렇지 않을 수도 있다. 

최종 도착지는 
- 극솟값 (local minimum)
- 최솟값 (global minimum)
- 안장값 (saddle point) 

일 수 있다. 

복잡하고 찌그러진 함수는 평평한 곳으로 파고들며 plateau-ing (학습 이뤄지지 않는 정체기) 할 수도 있다. 


경사하강법을 공식으로 나타내면 다음과 같다. 

<img src="https://t1.daumcdn.net/cfile/tistory/9912EE395A9292C918">

에타는 갱신하는 양, 즉 학습률(learning rate)을 나타낸다. 
이러한 매개변수를 하이퍼파라미터(hyper parameter)라고 부른다. 
가중치(w) 매개변수 같은 것들은 딥러닝에 의해 알아서 선택되는 반면, 하이퍼파라미터는 사람이 직접 설정해줘야 한다는 차이가 있다. 

학습률은 적당해야 한다. 너무 크면 너무 느리거나/local minimum에 빠지고 또는 너무 커서 overshooting 등의 문제가 발생할 수 있다.  

이를 간단히 구현해보자. 

In [20]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num): # step_num만큼 100번 움직인다. 
        grad = numerical_gradient(f, x)
        x = x -  lr * grad
    return x

In [21]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100) #거의 0에 가까운 결과 나온다. Jupyter에선 0. 으로 근사하여 나온다. 

array([-6.11110793e-10,  8.14814391e-10])

In [22]:
# 학습률이 너무 크거나 작다면? 

print(gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100)) # 너무 작을 때
print(gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)) # 너무 클 때 

[-6.11110781e-10  8.14814374e-10]
[ 3.18122051e+12 -3.96419374e+12]


### 4.4.2 신경망에서의 기울기 

a비용함수/a가중치 = aL/aW 를 구해야 한다. 

aL/aW의 shape는 W의 shape와 같다. (당연하지. L을 각각의 W로 편미분하는 것이니까.)

간단한 신경망에서 기울기를 구해보자. 

In [25]:
from common.functions import softmax, cross_entropy_error # 저자 폴더에 깔려있는 것임. 
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화. 
    
    def predict(self, X):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [26]:
net = simpleNet()
print(net.W)

[[ 2.06977105 -0.87160647 -0.55295386]
 [-0.70216698 -0.24967897 -0.21710586]]


In [27]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[ 0.60991235 -0.74767495 -0.52716759]


In [28]:
np.argmax(p) # 최대값의 인덱스 

0

In [29]:
t = np.array([0, 0, 1]) # 정답 레이블
net.loss(x, t)

1.5932598822609763

In [30]:
def f(W): # 손실함수를 계산. 
    return net.loss(x, t)

dW = numerical_gradient(f, net.W) # f는 함수, 두 번째 파라미터는 그 함수의 인수. 
print(dW)

[[ 0.38021951  0.09782317 -0.47804267]
 [ 0.57032926  0.14673475 -0.71706401]]


In [31]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W) # 간단한 함수이니 lambda 쓸 수도 있음. 
print(dW)

[[ 0.38021951  0.09782317 -0.47804267]
 [ 0.57032926  0.14673475 -0.71706401]]


## 4.5 학습 알고리즘 구현하기

신경망 학습의 절차를 다시 한 번 정리해보자. 

1. 전제

    신경망에는 적용 가능한 가중치와 편향이 있고, 이 weight와 bias를 훈련데이터에 적응하도록 조정하는 과정을 '학습'이라 한다. 신경망 학습은 다음과 같은 4단계로 수행한다. 

2. 1단계: 미니배치

    훈련 데이터 중 일부를 무작위로 가져옴. 이 데이터를 미니 배치라 하며, 이 미니 배치의 loss function을 최소화 시키는 것이 목표. 
    
3. 2단계: 기울기 산출

    loss function에 대한 각 가중치 매개변수의 기울기를 구함. 기울기는 gradient descent 방법에 따라 손실함수 최소화의 방향을 제시함. 
    
4. 3단계: 매개변수 갱신

    가중치 매개변수를 기울기 방향으로 조금 갱신한다. (learning rate)
    
5. 4단계: 1~3을 반복한다. 


이 때, 미니배치를 무작위로 선택하기 때문에 이를 확률적 경사 하강법(Stochastic Gradient Descent, SGD)라고 한다. 